In [1]:
import base64
import datetime
import json
import os
import random
import time

import scipy
from scipy import ndimage
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels as sm
from pylab import rcParams
from pylab import *
from matplotlib.dates import date2num , DateFormatter
from PIL import Image

np.random.seed(1337)

%matplotlib inline
sns.set(font_scale=1.0)
rcParams['figure.figsize'] = 8, 6
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

pd.options.display.max_colwidth = 1000

In [28]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy import sparse

# Load the Data

In [3]:
d = pd.concat([pd.read_csv('tmp/train_ids.csv', index_col='id'),
               pd.read_csv('tmp/test_ids.csv', index_col='id')])

In [4]:
e = pd.read_csv('tmp/features_bathrooms.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [5]:
e = pd.read_csv('tmp/features_bedrooms.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [6]:
e = pd.read_csv('tmp/features_building_id.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [7]:
e = pd.read_csv('tmp/features_created.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [8]:
e = pd.read_csv('tmp/features_description.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [9]:
e = pd.read_csv('tmp/features_display_address.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [10]:
e = pd.read_csv('tmp/features_features.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [11]:
e = pd.read_csv('tmp/features_latlon.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [12]:
e = pd.read_csv('tmp/features_manager_id.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [13]:
e = pd.read_csv('tmp/features_photos.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [14]:
e = pd.read_csv('tmp/features_price.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [15]:
e = pd.read_csv('tmp/features_street_address.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [16]:
e = pd.read_csv('tmp/features_others.csv', index_col='id')
d = pd.merge(d, e, how='left', left_index=True, right_index=True)

In [17]:
d.interest_level.value_counts()

low       34284
medium    11229
high       3839
Name: interest_level, dtype: int64

In [18]:
del d['num_apts_in_building_q10']
del d['num_apts_in_building']

In [19]:
# d = pd.concat([d, tf_idfs], axis=1)
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124011 entries, 0 to 124010
Columns: 108 entries, listing_id to hours_since_managers_last_listing
dtypes: float64(52), int64(44), object(12)
memory usage: 103.1+ MB


In [20]:
tr = d[d.interest_level.notnull()].copy()
print len(tr)

49352


In [21]:
trl = tr.interest_level.copy()
trf = tr
del trf['interest_level']

In [22]:
te = d[d.interest_level.isnull()].copy()
print len(te)

74659


In [23]:
tef = te
del tef['interest_level']

In [24]:

tfidf_features = CountVectorizer(stop_words='english', max_features=400)
tr_sparse_features = tfidf_features.fit_transform(trf["features"])
te_sparse_features = tfidf_features.transform(tef["features"])

tfidf_desc = CountVectorizer(stop_words='english', max_features=400)
tr_sparse_desc = tfidf_desc.fit_transform(trf["desc_clean"].fillna(""))
te_sparse_desc = tfidf_desc.transform(tef["desc_clean"].fillna(""))

In [25]:
for c in trf.columns:
    if trf.dtypes[c] not in ('int64', 'float64'):
        del trf[c]           
            
for c in tef.columns:
    if tef.dtypes[c] not in ('int64', 'float64'):
        del tef[c]

In [26]:
for c in trf.columns:
    trf[c] = trf[c].fillna(0)
for c in tef.columns:
    tef[c] = tef[c].fillna(0)

# Cross validation

In [30]:
train_X = sparse.hstack([trf,tr_sparse_features, tr_sparse_desc]).tocsr()
test_X = sparse.hstack([tef,te_sparse_features, te_sparse_desc]).tocsr()

target_num_map = {'high':0, 'medium':1, 'low':2}
train_y = np.array(trl.apply(lambda x: target_num_map[x]))
train_y

array([1, 2, 1, ..., 1, 1, 0])

In [96]:

# class sklearn.tree.DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_split=1e-07, class_weight=None, presort=False)
clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy', max_leaf_nodes=6),
                         n_estimators=1000, 
                         learning_rate=0.5, 
                         algorithm='SAMME.R',
                         random_state=1)
scores = cross_val_score(clf, train_X, train_y, scoring='neg_log_loss')

In [97]:
print scores

[-1.0728669  -1.07891689 -1.07669291]


# Run classifier

In [31]:
clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy', max_leaf_nodes=6),
                         n_estimators=1000, 
                         learning_rate=0.5, 
                         algorithm='SAMME.R',
                         random_state=1)
clf = clf.fit(train_X, train_y)


In [32]:
preds = clf.predict_proba(test_X)

In [33]:
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = pd.read_csv('input/sample_submission.csv').listing_id.values
out_df.to_csv("output/adaboost_1.csv", index=False)